In [2]:
import dask
from distributed import Client, LocalCluster
from daskqueue.Protocol import Message
from time import perf_counter

In [3]:
func = lambda x: x + 2

def rdx_msg():
    msg = Message(func, 12)
    return msg


In [5]:
cluster= LocalCluster(
    n_workers=6,
    threads_per_worker=1,
    dashboard_address=":3338",
    memory_limit="2GB"
)

2023-01-02 18:41:10,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-64juugzz', purging
2023-01-02 18:41:10,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-jxt3gv4c', purging
2023-01-02 18:41:10,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-_4rzcgcc', purging
2023-01-02 18:41:10,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-l335dt48', purging
2023-01-02 18:41:10,412 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-cs8k9qde', purging


In [6]:
client = Client(cluster,direct_to_workers=True)

In [7]:
from daskqueue import ConsumerPool, Durability, QueuePool
import tempfile

no_func = lambda: None

In [8]:
N = 10_000
n_queues = 1
n_consumers = 2

with tempfile.TemporaryDirectory() as tmpdirname:
    s = perf_counter()
    queue_pool = QueuePool(
        client, n_queues, durability=Durability.DURABLE, dirpath=str(tmpdirname)
    )

    for _ in range(N):
        queue_pool.submit(no_func)
    e = perf_counter()
    bs_ops = N / (e - s)

2023-01-02 18:41:34,535,535 INFO: Created 1 queues in Cluster and one QueueManager.


In [9]:
bs_ops

544.702293690951

In [10]:
client.restart()

2023-01-02 18:42:18,158 - distributed.nanny - WARNING - Restarting worker
2023-01-02 18:42:18,202 - distributed.nanny - WARNING - Restarting worker
2023-01-02 18:42:18,212 - distributed.nanny - WARNING - Restarting worker
2023-01-02 18:42:18,217 - distributed.nanny - WARNING - Restarting worker
2023-01-02 18:42:18,235 - distributed.nanny - WARNING - Restarting worker
2023-01-02 18:42:18,240 - distributed.nanny - WARNING - Restarting worker


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:3338/status,
Dashboard: http://127.0.0.1:3338/status,Workers: 6
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36223,Workers: 6
Dashboard: http://127.0.0.1:3338/status,Total threads: 6
Started: 1 minute ago,Total memory: 11.18 GiB
Comm: tcp://127.0.0.1:41135,Total threads: 1
Dashboard: http://127.0.0.1:37791/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:45737,


In [12]:
N = 1_000_000

In [14]:
class Counter:
    n = 0

    def __init__(self):
        self.n = 0

    def increment(self):
        self.n += 1
        return self.n
    def add(self, x):
        self.n += x
        return self.n

future = client.submit(Counter, actor=True)  # Create a Counter on a worker
counter = future.result()  


In [15]:
s = perf_counter()
for _ in range(N):
    future = counter.increment()                 # Call remote method
    # res = future.result() 
e = perf_counter()
wops = N / (e - s)

In [19]:
wops

44666.60182065287

2023-01-02 18:56:12,133 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-01-02 18:56:18,782 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-01-02 18:56:23,719 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
